# Vector Search Testing

In [1]:
from DocumentDBVectorOperations import DocumentDBVectorSearch
from Chunking import chunking, combine_text
from DocumentLoader import load_and_read_pdf
from GenerateEmbeddings import generate_embeddings

In [2]:
docdb_obj =  DocumentDBVectorSearch()
docdb_obj.establish_connection()
docdb_obj.create_index()

Make sure you are running ssh tunnel before executing this function


In [3]:
pages = load_and_read_pdf(file_path = "VisionTransformers.pdf")

final_text = combine_text(pages)

chunks_of_text = chunking(final_text)

In [6]:
docdb_obj.add_texts(chunks_of_text)

Documents inserted successfully.


In [4]:
prompt = "What is Vision Transformers?"
results = docdb_obj.vector_search(prompt, num_results=4)
for cur_text in results:
    print(cur_text['text'])
    print("=========================")

and a pure transformer applied directly to sequences of image patches can perform
very well on image classiﬁcation tasks. When pre-trained on large amounts of
data and transferred to multiple mid-sized or small image recognition benchmarks
(ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains excellent
results compared to state-of-the-art convolutional networks while requiring sub-
stantially fewer computational resources to train.1
1 I NTRODUCTION
Self-attention-based architectures, in particular Transformers (Vaswani et al., 2017), have become
the model of choice in natural language processing (NLP). The dominant approach is to pre-train on
a large text corpus and then ﬁne-tune on a smaller task-speciﬁc dataset (Devlin et al., 2019). Thanks
to Transformers’ computational efﬁciency and scalability, it has become possible to train models of
unprecedented size, with over 100B parameters (Brown et al., 2020; Lepikhin et al., 2020). With the
Transformers appear not to satura

# RAG Pipeline Testing

In [1]:
from DocumentDBVectorOperations import DocumentDBVectorSearch
docdb_obj =  DocumentDBVectorSearch()
docdb_obj.establish_connection()
docdb_obj.create_index()

Make sure you are running ssh tunnel before executing this function


In [2]:
groq_api_key = "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"

In [3]:
from RagPipeline import RAGPipeline

In [4]:
rag_obj = RAGPipeline(groq_api_key=groq_api_key, Docdb_VS=docdb_obj)

In [5]:
answer = rag_obj.generate_answer("What is Vision Transformer? give answer in single paragraph", 3)

In [6]:
print(answer)

A Vision Transformer (ViT) is a type of transformer architecture that is applied directly to sequences of image patches to perform image classification tasks. It works by first linearly projecting the flattened patches into a lower-dimensional space, then adding a learned position embedding to the patch representations. The model learns to encode distance within the image in the similarity of position embeddings, allowing it to understand the structure of the image. When pre-trained on large amounts of data and transferred to multiple image recognition benchmarks, ViT attains excellent results compared to state-of-the-art convolutional networks while requiring substantially fewer computational resources to train, making it a highly efficient and scalable architecture for computer vision tasks.


In [17]:
vs_results = docdb_obj.vector_search("What is vision transformers?")

In [19]:
for result in vs_results:
    print(result['text'])

and a pure transformer applied directly to sequences of image patches can perform
very well on image classiﬁcation tasks. When pre-trained on large amounts of
data and transferred to multiple mid-sized or small image recognition benchmarks
(ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains excellent
results compared to state-of-the-art convolutional networks while requiring sub-
stantially fewer computational resources to train.1
1 I NTRODUCTION
Self-attention-based architectures, in particular Transformers (Vaswani et al., 2017), have become
the model of choice in natural language processing (NLP). The dominant approach is to pre-train on
a large text corpus and then ﬁne-tune on a smaller task-speciﬁc dataset (Devlin et al., 2019). Thanks
to Transformers’ computational efﬁciency and scalability, it has become possible to train models of
unprecedented size, with over 100B parameters (Brown et al., 2020; Lepikhin et al., 2020). With the
Transformers appear not to satura